In [ ]:
import json
import pandas as pd
import os

model_names = [
    "claude-3-5-sonnet",
    "flash",
    "gpt-4o-mini",
    "gpt4o",
    "pro"
]

# Dictionary to hold results keyed by video_name
video_dict = {}

for name in model_names:
    file_path = f"responses_{name}_cot_1203.json"
    if not os.path.exists(file_path):
        continue

    with open(file_path, "r") as f:
        data = json.load(f)

    for item in data:
        for video_name, content in item.items():
            if video_name not in video_dict:
                video_dict[video_name] = {}

            if isinstance(content, str):
                try:
                    content_json = json.loads(content)
                    video_dict[video_name][f"{name}-description"] = content_json.get("video_description", "")
                    video_dict[video_name][f"{name}-reasoning"] = content_json.get("reasoning", "")
                    video_dict[video_name][f"{name}-anomaly"] = content_json.get("anomaly", "")
                except json.JSONDecodeError:
                    video_dict[video_name][f"{name}-description"] = ""
                    video_dict[video_name][f"{name}-reasoning"] = ""
                    video_dict[video_name][f"{name}-anomaly"] = ""
            else:
                video_dict[video_name][f"{name}-description"] = ""
                video_dict[video_name][f"{name}-reasoning"] = ""
                video_dict[video_name][f"{name}-anomaly"] = ""

# Convert to DataFrame
df = pd.DataFrame.from_dict(video_dict, orient='index')
df.index.name = "video_name"
df.reset_index(inplace=True)

# Save to Excel
df.to_excel("extracted_video_anomalies_all_models.xlsx", index=False)


In [ ]:
import pandas as pd
import os

# =========================
# Paths (new files)
# =========================
groundtruth_path = "Annotation_vad_1203.csv"
predictions_path = "extracted_video_anomalies_all_models.xlsx"

# =========================
# Load files
# =========================
groundtruth_df = pd.read_csv(groundtruth_path)
predictions_df = pd.read_excel(predictions_path, sheet_name=0)  # first sheet

# =========================
# Normalize IDs / filenames
# =========================
# Ground truth: use Title as the identifier
groundtruth_df["Video Name"] = (
    groundtruth_df["Title"].astype(str).str.strip()
)

# True label from ground truth
groundtruth_df["True Label"] = groundtruth_df["Label"].astype(str).str.strip()

# Optional: map ground truth to binary anomaly label (Normal=0, others=1)
groundtruth_df["True Anomaly"] = groundtruth_df["True Label"].map(
    lambda x: 0 if str(x).strip().lower() == "normal" else 1
)

# Predictions: normalize video_name by removing extension
predictions_df["Video Name"] = (
    predictions_df["video_name"].astype(str).str.strip()
    .str.replace(".mp4", "", regex=False)
    .str.replace(".avi", "", regex=False)
    .str.replace(".mov", "", regex=False)
)

# =========================
# Model columns in the new Excel
# =========================
model_suffixes = {
    "claude-3-5-sonnet": "claude-3-5-sonnet-anomaly",
    "flash": "flash-anomaly",
    "gpt-4o-mini": "gpt-4o-mini-anomaly",
    "gpt4o": "gpt4o-anomaly",
    "pro": "pro-anomaly",
}

# Output directory
output_dir = "model_predictions"
os.makedirs(output_dir, exist_ok=True)

# =========================
# Helper: merge with fallback
# =========================
gt_keep = groundtruth_df[["Video Name", "True Label", "True Anomaly"]].copy()

def merge_with_fallback(model_df: pd.DataFrame, gt_df: pd.DataFrame) -> pd.DataFrame:
    # 1) try merge by Video Name
    merged = pd.merge(model_df, gt_df, on="Video Name", how="left")

    match_rate = merged["True Label"].notna().mean()
    print(f"[INFO] Merge-by-name match rate: {match_rate:.2%}")

    # 2) fallback: align by row order if almost nothing matches
    if match_rate < 0.05:
        print("[WARN] Very low name match. Falling back to row-order alignment.")
        a = model_df.reset_index(drop=True).copy()
        b = gt_df.reset_index(drop=True).copy()

        if len(a) != len(b):
            raise ValueError(
                f"Row-order fallback failed: predictions rows={len(a)} vs groundtruth rows={len(b)}"
            )

        merged = pd.concat([a, b[["True Label", "True Anomaly"]]], axis=1)

    return merged

# =========================
# Process and save CSVs
# =========================
for model_name, col_name in model_suffixes.items():
    if col_name not in predictions_df.columns:
        print(f"[WARN] Column missing for {model_name}: {col_name}. Skipping.")
        continue

    model_df = predictions_df[["Video Name", col_name]].rename(columns={
        col_name: "Predicted Anomaly"
    })

    merged = merge_with_fallback(model_df, gt_keep)

    out_path = os.path.join(f"vad_results_{model_name}.csv")
    merged.to_csv(out_path, index=False)
    print(f"[OK] Saved: {out_path}")


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

# Load Excel file
file_path = "extracted_video_anomalies_all_models.xlsx"
df = pd.read_excel(file_path)

# Initialize model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Define model names
model_names = ["claude-3-5-sonnet", "flash", "gpt-4o-mini", "gpt4o", "pro"]

# Prepare final DataFrame
final_data = pd.DataFrame()
final_data["video_name"] = df["video_name"]

# Embed and extract selected fields
for model_name in model_names:
    desc_col = f"{model_name}-description"
    reas_col = f"{model_name}-reasoning"
    anomaly_col = f"{model_name}-anomaly"

    if desc_col in df.columns:
        desc_texts = df[desc_col].tolist()
        desc_embeds = []
        for text in desc_texts:
            if pd.isna(text):
                desc_embeds.append(np.nan)
            else:
                desc_embeds.append(model.encode(text))
        final_data[f"{model_name}-desc_emb"] = desc_embeds

    if reas_col in df.columns:
        reas_texts = df[reas_col].tolist()
        reas_embeds = []
        for text in reas_texts:
            if pd.isna(text):
                reas_embeds.append(np.nan)
            else:
                reas_embeds.append(model.encode(text))
        final_data[f"{model_name}-reas_emb"] = reas_embeds

    if anomaly_col in df.columns:
        final_data[f"{model_name}-anomaly"] = df[anomaly_col]

# Save to output file
output_path = "embedded_only_video_anomalies.xlsx"
final_data.to_excel(output_path, index=False)

print(f"✅ Done! File saved as: {output_path}")
